In [ ]:
import pypsa
import xarray as xr
import hvplot.xarray
import pandas as pd
import hvplot.pandas
import numpy as np

import seaborn

import re
from pathlib import Path

import logging

logger = logging.getLogger(__name__)

from _helpers import configure_logging
configure_logging(snakemake)


seaborn.set_style("whitegrid")

## +++ Read and combine all 'results.csv' located in subdirectories of the scenario
fp = Path(snakemake.input["scenario_dir"])
fps = list(fp.rglob("results.csv"))

logger.info(f"Found {len(fps)} 'results.csv' files in subdirectories. Trying to read all of them.")

df = pd.concat([pd.read_csv(f, sep=";") for f in fps])


## +++ Only consider LCoE here
lcoe = df.query("subcategory == 'Cost per MWh delivered'")

lcoe = lcoe[["year","exporter","esc","value"]]


fig_overview = (
    lcoe.hvplot.points(x='year', y='value', padding=0.2, hover_cols='all', by="esc", title="LCoE overview", ylabel="LCoE [EUR/MWh]")
    * lcoe.hvplot.labels(x='year', y='value', text='exporter', text_baseline='bottom', text_align="left", hover=False, text_font_size="small")
)

fig_lcoe_development = lcoe.hvplot.bar(x="esc", y="value", by="exporter", alpha=.3, hover_cols=["value","year"], ylabel="LCoE [EUR/MWh]", title="LCoE development by year")

## +++ Two types of plots showing the LCoE distribution (box plot) and development by exporter (line+markers)
# Plots are created as singular plots per ESC and later stitched together as creating them via automatic
# subplots=True with hvplots failed
figs_lcoe_distribution = []
figs_lcoe_by_exporter = []

for esc in lcoe["esc"].unique():
    _lcoe = lcoe[lcoe["esc"]==esc]

    ## +++ LCoE distribution (box) plot per ESC and year, statistics on all exporters. 
    figs_lcoe_distribution.append(
        _lcoe.hvplot.box(by="year", y="value", ylabel="LCoE [EUR/MWh]", title=f"Cost distribution: {esc}")
        * _lcoe.hvplot.scatter(x="year", y="value", c="black", hover_cols=["exporter","value"])
    )
    
    ## +++ Figure with overview and connected lines (per exporter) of LCoE development with years
    figs_lcoe_by_exporter.append(
        _lcoe.hvplot.scatter(y="value", x="year", by="exporter", title=f"LCoE by exporter: {esc}")
        * _lcoe.hvplot.line(y="value", x="year", by="exporter", title=f"LCoE by exporter: {esc}")
    )

def combine_hvplots_from_list(fig_list):
    """Combine a list of hvplot plots into a single plot by stitching (adding) them together."""
    
    combined_fig = None
    for fig in fig_list:

        combined_fig = combined_fig + fig if combined_fig is not None else fig

    return combined_fig

figs_lcoe_by_exporter = combine_hvplots_from_list(figs_lcoe_by_exporter)
figs_lcoe_distribution = combine_hvplots_from_list(figs_lcoe_distribution)

## +++ Combine all plots into one
# Cols: have one row per plot types (all ESCs in one row)
figs = (
    figs_lcoe_by_exporter 
    + figs_lcoe_distribution 
    + fig_lcoe_development
    + fig_overview 
).cols(len(df["esc"].unique()))

## +++ Save figures
for fp in snakemake.output["fig"]:
    hvplot.save(figs, fp)